# Teil 4 Demo 2: Abfragen mit Tidyverse

In dieser Demo wird gezeigt, wie Medizinische Beispieldaten (aus der MIMIC III-Datenbank) mit Tidyverse-Befehlen im Speicher abgefragt werden können. Die Abfragen haben dabei jeweils das gleiche Ziel wie Demo 1 dieses Abschnitts, werden aber statt mit SQL mit Tidyverse-Befehlen im Speicher ausgeführt. Die Erläuterungen beschränken sich in diesem Teil deshalb auf die Besonderheiten der Tidyverse-Befehle (ggf. für Details in Demo 1 nachschauen).

## Bibliotheken & Konfiguration

In diesem Abschnitt werden benötigte Programmpakete geladen und Konfigurationsvariablen z.B. für die Datenquellen gesetzt.

Hinweis: auf Google Colab kann das Laden der Pakete vor allem beim ersten Aufruf einige Minuten dauern. Bitte führen Sie diesen Block dann nicht erneut aus, sondern warten die Ausführung ab.

In [ ]:
packages <- c("readr", "dplyr", "stringr", "tidyr")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-Datenanalyse-Visualisierung/master/Rohdaten/mimic-iii-demo/"

## Ausgewählte MIMIC III-Rohdaten laden

Wie in Demo 1 laden wir zunächst 3 Tabellen des MIMIC III-Datensatzes in Dataframes (im Tidyverse auch "Tibbles" genannt). Anders als in der vorherigen Demo müssen sie nicht in eine Datenbank geladen werden, sondern können direkt im Speicher weiterverarbeitet werden.

In [ ]:
# Patientenstammdaten laden
mimic.patients.raw <- read_delim(paste0(base_url, "PATIENTS.csv"),
                                 col_types = cols(row_id = col_double(), subject_id = col_double(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()),
                                 skip = 0, delim = ",")

# Behandlungsfälle laden
mimic.admissions.raw <- read_delim(paste(base_url, "ADMISSIONS.csv", sep=""),
                                   col_types = cols(  row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()),
                                   skip = 0, delim = ",")
# Verschreibungen laden
mimic.prescriptions.raw <- read_delim(paste(base_url, "PRESCRIPTIONS.csv", sep=""),
                                      col_types = cols(row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), icustay_id = col_double(), startdate = col_datetime(format = ""), enddate = col_datetime(format = ""), drug_type = col_character(), drug = col_character(), drug_name_poe = col_character(), drug_name_generic = col_character(), formulary_drug_cd = col_character(), gsn = col_character(), ndc = col_character(), prod_strength = col_character(), dose_val_rx = col_character(), dose_unit_rx = col_character(), form_val_disp = col_character(), form_unit_disp = col_character(), route = col_character()),
                                      skip = 0, delim = ",")


## Einfache Abfragen auf Rohdaten

### Alle Spalten eines Tibble abfragen

Um vergleichbar zu `SELECT * FROM <table>` alle Spalten & Zeilen eines Dataframe anzuzeigen, muss hier nur der Dataframe angegeben werden. Der `head()`-Befehl schränkt die Anzeige wie zuvor auf die ersten 6 Zeilen ein.

In [ ]:
head(mimic.patients.raw)

### Ausgewählte Spalten eines Tibble abfragen

Über das "Chaining"-Symbol "%>%", das einen Datenfluss von links nach rechts darstellen soll, können im Tidyverse beliebig viele Verarbeitungsschritte miteinander verkettet werden. In diesem Fall wenden wir die `select()`-Funktion an, um nur benannte Spalten aus dem Dataframe abzufragen.

In [ ]:
head(mimic.patients.raw %>% select(subject_id, gender))

### Ausgewählte Zeilen eines Tibble abfragen

Mit der `filter()`-Funktion können Datensätze selektiert werden.

**Merke** **Text fett markieren**: bei Vergleichen muss das doppelte Gleichheitszeichen (==) verwendet werden!

In [ ]:
head(mimic.patients.raw %>% filter(gender == 'M'))

### Ergebnis auf eindeutige Datensätze reduzieren (DISTINCT)

Wie in SQL kann die `distinct()`-Funktion verwendet werden, um die Ergebnismenge auf eindeutige Ausprägungen zu reduzieren.

In [ ]:
mimic.patients.raw %>%
    select(gender) %>%
    distinct()

## Daten aggregieren

### Einfache Aggregation: Gesamtzahl der Datensätze eines Tibble abfragen

Mit der `summarize()`-Funktion können Daten im Tidyverse aggregiert werden. Gegenstück zur COUNT(*)-Funktion ist hierbei die Funktion `n()`, mit der die Anzahl der Datensätze ermittelt wird.

In [ ]:
mimic.patients.raw %>% summarize(n = n())

### Aggregation & Gruppierung: Anzahl nach Geschlecht ermitteln

Auch im Tidyverse ist die Gruppierung von Datensätzen bei der Aggregation mit der `group_by()`-Funktion möglich.

In [ ]:
mimic.patients.raw %>%
    group_by(gender) %>%
    summarize(n = n(), .groups="keep")

# Merke: die Option .groups="keep" ist nötig, damit das Gruppierungsmerkmal in der Ausgabe übernommen wird!

### Zeilen nach der Aggregation filtern

Während in SQL die `HAVING`-Klausel notwendig ist, um nach der Aggregation zu selektieren, kann im Tidyverse die normale `filter()`-Funktion verwendet werden, wird aber in der Pipeline hinter die Aggregation gestellt.

In [ ]:
mimic.patients.raw %>%
    group_by(gender) %>%
    summarize(n = n(), .groups="keep") %>%
    filter(n > 50)

## Tabellen verknüpfen

### Tibbles per JOIN miteinander verknüpfen

Im Tidyverse ist `inner_join()` die Entsprechung der `JOIN`-Klausel aus SQL. mit dem `by`-Attribut werden die (hier gleich benannten) Spalten zur Verknüpfung der beiden Dataframes angegeben.

In [ ]:
head(mimic.patients.raw %>%
     inner_join(mimic.admissions.raw, by = "subject_id") %>%
     select(subject_id, gender, diagnosis)
)

### Patient:innen mit mehr als einem Intensiv-Aufenthalt abfragen

Auch hier kann nach dem JOIN-Schritt mit den üblichen Filterungs, Gruppierungs- und Aggregatfunktionen gearbeitet werden.

In [ ]:
head(mimic.patients.raw %>%
     inner_join(mimic.admissions.raw, by = "subject_id") %>%
     group_by(subject_id) %>%
     summarize(n = n(), .groups="keep") %>%
     filter(n > 1)
)

## Komplexe Abfragen

### 2 Subsets von Verschreibungen für die Demo verschiedener Joins erzeugen

* Patient:innenen mit Hauptdiagnose Sepsis
* Patient:innen mit Gabe von Vancomycin (Reserve-Antibiotikum z.B. bei MRSA)

In [ ]:
demo.sepsis     <- mimic.admissions.raw %>%
                        filter(str_detect(tolower(diagnosis), "sepsis")) %>%
                        select(subject_id) %>%
                        distinct()
demo.vancomycin <- mimic.prescriptions.raw %>%
                        filter(str_detect(tolower(drug), "vancomycin")) %>%
                        select(subject_id) %>%
                        distinct()

### Patient:innen abfragen, die sowohl eine Sepsis als Hauptdiagnose hatten als auch Vancomycin erhalten haben (INNER JOIN)

Wieder verknüpfen wir die beiden Subsets, um Patienten zu ermitteln, die sowohl eine Sepsis als auch eine Vancomycin-Gabe hatten. Damit beide für den JOIN verwendeten Spalten erhalten bleiben, müssen wir das Attribut `keep=TRUE` angeben, da die (gleich benannten) Spalten sonst zusammengeführt würden. Da Spalten mit gleichen Namen im Tidyverse nicht erlaubt sind, müssen wir über das `suffix`-Attribut angeben, wie die Spalten je nach ihrer Quelltabelle eindeutig benannt werden sollen.

In [ ]:
demo.sepsis %>%
    inner_join(demo.vancomycin, by = "subject_id", keep=TRUE, suffix = c("_sepsis", "_vanco"))

### Alle Patient:innen abfragen, die eine Sepsis als Hauptdiagnose hatten sowie die mit Vancomycingabe ergänzen (LEFT OUTER JOIN)

Wie in SQL können neben INNER JOINs auch LEFT OUTER JOINs durchgeführt werden.

In [ ]:
demo.sepsis %>%
    left_join(demo.vancomycin, by = "subject_id", keep=TRUE, suffix = c("_sepsis", "_vanco"))

### Alle Patient:innen abfragen, die eine Vancomycingabe hatten und die keine Sepsisdiagnose hatten (RIGHT OUTER JOIN)

Im Tidyverse werden (anders als in SQLite) RIGHT OUTER JOINS mit einem eigenen Befehl unterstützt, so dass wir hier *nicht* als Workaround die Reihenfolge der Tabellen vertauschen müssen.

In [ ]:
head(demo.sepsis %>%
     right_join(demo.vancomycin, by = "subject_id", keep=TRUE, suffix = c("_sepsis", "_vanco")),
15)

### Tatsächliche Hauptdiagnosen für die Patient:innen mit Vancomycingabe ohne Sepsis ermitteln

Im abschließenden Statement kombinieren wir 3 Tabellen, um für die Patienten mit Vancomycingabe ohne Sepsis die tatsächlichen Haupdiagnosen zählen zu können.

In [ ]:
head(demo.vancomycin %>%
     inner_join(mimic.admissions.raw, by = "subject_id") %>%
     left_join(demo.sepsis, by = "subject_id", keep=TRUE, suffix=c("", "_sepsis")) %>%
     filter(is.na(subject_id_sepsis)) %>%
     group_by(diagnosis) %>%
     summarize(n = n(), .groups="keep") %>%
     arrange(desc(n))
, 10)